In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs A - {today}"

In [4]:
session_id = '2110944'

In [5]:
task =  "Consider the first week of August 2020 - find the names of 3 pitchers who's curveballs were most similar to Max Scherzer's."
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Retrieve Node ***

Distance to neareast plan: 0.5695837736129761

*** Entered Initialize Node ***

Formulating a new plan to solve the task.


IndexError: list index out of range

In [ ]:
task = """
make sure the plan follows this general flow:
1) create an average vector for each pitcher's curveball.
2) Train a knn model on this data
3) Use this model to find the 3 pitchers with the most similar curveball to Max Scherzer
"""
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'approved'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)

In [ ]:
task = 'yes'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)